#   [Py4Hw User Guide](../UserGuide.ipynb)> 11.1 HLS of combinational circuits

High Level Synthesis in py4hw refers to the process of creating hardware from python source code. The result of this process is not a Verilog code, but a py4hw structural model, which can then be translated to verilog.

Thus, this is a source-to-source transformation in the python language domain.

Let's create this circuit to compute the square root of an integer value using the babylonian method.
Note that is is a combinational circuit, because all the behavior takes place in the propagate method. 

In [1]:
import py4hw

In [2]:
class SquareRoot(py4hw.Logic):
    def __init__(self, parent, name, a, r):
        super().__init__(parent, name)
        
        self.a = self.addIn('a', a)
        self.r = self.addOut('r', r)
        
        
    def propagate(self):
        maxv = 1 << self.a.getWidth()
        maxiter = int(math.log2(math.log2(maxv))) + 4
        #print('max iters:', maxiter)
        
        n = self.a.get() 
        if n == 0:
            r = 0
        elif n == 1:
            r = 1
        else:
            guess = n
            done = False
            for _ in range(maxiter):
                # For a combinational circuit loops must be fully unrollable,
                # otherwise no synthesys is possible
                if (done):
                    pass
                else:
                    next_guess = (guess + (n // guess)) // 2  # Integer division

                    if next_guess >= guess:  # Convergence condition: guess stops decreasing
                        done = True
                    else:
                        guess = next_guess
            r = guess
            
        self.r.put(r)


we want to create an structural circuit from this behavioral description. This is what we expect from High Level Synthesis workflows. However, typical HLS workflows will create Verilog from high-level language. We can follow a different approach, generate a py4hw structural model (in Python) from a py4hw behavioral model (also in Python, of course).  

In [3]:
from py4hw.transpilation.python2structural import Python2StructuralCode

In [4]:
coder = Python2StructuralCode(SquareRoot)

print(coder.getCode())

# This file was automatically created by Python2StructuralCode
class SquareRootStructural(Logic):

    def __init__(self, parent, name, a, r):
        super().__init__(parent, name)
        self.a = self.addIn('a', a)
        self.r = self.addOut('r', r)
        self.build()

    def build(self):
        maxv = 1 << self.a.getWidth()
        maxiter = int(math.log2(math.log2(maxv))) + 4
        n = self.a.get()
        if n == 0:
            r = 0
        elif n == 1:
            r = 1
        else:
            guess = n
            done = False
            for _ in range(maxiter):
                if done:
                    pass
                else:
                    next_guess = (guess + n // guess) // 2
                    if next_guess >= guess:
                        done = True
                    else:
                        guess = next_guess
            r = guess
        self.r.put(r)


In [5]:
coder.clazz.__name__

'SquareRoot'